# MESSAGE DECODING FROM UNPOOLED SAMPLES (BUT COMBINED INTO MOCK POOLS AND SUBSAMPLED)

### INPUT, LOAD SEQUENCES

In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import matplotlib as mpl
import scipy.stats

from turtles.turtles_utils import *

In [2]:
base_path = '/mnt/c/Users/jonst/Northwestern University/Tyo Lab - Shared group resources' \
            '/Publications Presentations & Proposals from Group/Manuscripts' \
            '/2022 - Callisto Strutz - tdt data storage'

#: Overall directory for TdT project data and analysis
tdt_dir = f'{base_path}/02 Analyzed data/'

#: Subdirectory for NGS run
data_dir = f'{base_path}/01 Raw data/20211109_barcoded_rec_1/data/rawDataSingles/'

#: Directory to save generated figures in
figure_dir = f'{base_path}/02 Analyzed data/preliminary_figures/Hello_World_1_subsampled/'

In [3]:
sample_max_n_reads = [300000, 100000, 10000, 1000, 100]
sample_n_random_states = [1, 5, 15, 30, 50]

In [4]:
#import conditions table
conditions_df = pd.DataFrame()
conditions_df = pd.read_excel(f'{base_path}/01 Raw data/20211109_barcoded_rec_1/analysis/conditonsTable.xls', header=0)
conditions_df

,Filename,Sample,Category,Replicate,Timepoint,Condition,Position,Base_Condition
0,A1,1C,c,1,1,1,NaN,0
1,A2,2C,c,2,1,1,NaN,0
2,A3,4C,c,3,1,1,NaN,0
3,A4,6C,c,1,1,3,NaN,gg
4,A5,9c,c,2,1,3,NaN,gg
5,A6,10C,c,3,1,3,NaN,gg
6,A7,15C,c,1,1,4,NaN,t
7,A8,16C,c,2,1,4,NaN,t
8,A9,17C,c,3,1,4,NaN,t
9,A10,18C,c,1,0,6,NaN,0


In [5]:
# split controls and message
controls_df = conditions_df[conditions_df['Category']=='c']
message_df = conditions_df[conditions_df['Category']=='r']

controls_df = controls_df.reindex(columns=['Filename', 'Sample', 'Replicate'])
message_df = message_df.reindex(columns=['Filename','Sample', 'Replicate'])

In [6]:
controls_df

,Filename,Sample,Replicate
0,A1,1C,1
1,A2,2C,2
2,A3,4C,3
3,A4,6C,1
4,A5,9c,2
5,A6,10C,3
6,A7,15C,1
7,A8,16C,2
8,A9,17C,3
9,A10,18C,1


In [7]:
message_df

,Filename,Sample,Replicate
24,C1,1R,1
25,C2,2R,1
26,C3,4R,1
27,C4,6R,1
28,C5,9R,1
29,C6,10R,1
30,C7,15R,1
31,C8,16R,1
32,C9,17R,1
33,C10,18R,1


In [8]:
# hgRNA reverse complements
hgRNA_dict_rc = {'D21': 'CTTGGCCGTAGCGTGAC'}

seqs_dict, hgRNAs, barcodes, insert_barcodes = read_in_vivo_seqs_R2(data_dir, hgRNA_dict_rc)


Read 107373 sequences in A1_S1_L001_R2_001.fastq.gz ...
Number cut and inserted into: 12887
Average length (excluding 0-length seqs): 2.37


Read 100379 sequences in A10_S10_L001_R2_001.fastq.gz ...
Number cut and inserted into: 304
Average length (excluding 0-length seqs): 1.11


Read 110847 sequences in A11_S11_L001_R2_001.fastq.gz ...
Number cut and inserted into: 562
Average length (excluding 0-length seqs): 1.18


Read 80528 sequences in A12_S12_L001_R2_001.fastq.gz ...
Number cut and inserted into: 407
Average length (excluding 0-length seqs): 1.16


Read 96780 sequences in A2_S2_L001_R2_001.fastq.gz ...
Number cut and inserted into: 12039
Average length (excluding 0-length seqs): 2.25


Read 89092 sequences in A3_S3_L001_R2_001.fastq.gz ...
Number cut and inserted into: 11871
Average length (excluding 0-length seqs): 2.52


Read 96479 sequences in A4_S4_L001_R2_001.fastq.gz ...
Number cut and inserted into: 7293
Average length (excluding 0-length seqs): 2.56


Read 115858 seque

# COMBINE READS INTO MOCK POOL

In [9]:
control_seqs = []
control_hgRNAs = []
control_barcodes = []
control_insert_barcodes = []

message_seqs = []
message_hgRNAs = []
message_barcodes = []
message_insert_barcodes = []

for condition in seqs_dict:
    file_seqs = seqs_dict[condition]
    file_hgRNAs = hgRNAs[condition]
    file_barcodes = barcodes[condition]
    file_insert_barcodes = insert_barcodes[condition]
    
    if 'A' in condition or 'B' in condition:
        control_seqs += file_seqs
        control_hgRNAs += file_hgRNAs
        control_barcodes += file_barcodes
        control_insert_barcodes += file_insert_barcodes
    elif 'C' in condition or 'D' in condition:
        message_seqs += file_seqs
        message_hgRNAs += file_hgRNAs
        message_barcodes += file_barcodes
        message_insert_barcodes += file_insert_barcodes

In [10]:
len(control_seqs)

152617

In [11]:
len(message_seqs)

244074

In [12]:
sample_max_n_reads

[300000, 100000, 10000, 1000, 100]

In [13]:
sample_n_random_states

[1, 5, 15, 30, 50]

In [14]:
seqs_dict = {}
hgRNAs = {}
barcodes = {}
insert_barcodes = {}

for max_n_reads, n_random_states in zip(sample_max_n_reads, sample_n_random_states):
    
    seqs_dict[max_n_reads] = []
    hgRNAs[max_n_reads] = []
    barcodes[max_n_reads] = []
    insert_barcodes[max_n_reads] = []

    for rs in range(n_random_states):
        
        sampled_control_seqs = []
        sampled_control_hgRNAs = []
        sampled_control_barcodes = []
        sampled_control_insert_barcodes = []
        
        sampled_message_seqs = []
        sampled_message_hgRNAs = []
        sampled_message_barcodes = []
        sampled_message_insert_barcodes = []
        
        random.seed(rs)
        
        control_indices = range(len(control_seqs))
        sampled_control_indices = random.sample(control_indices, min(max_n_reads, len(control_indices)))
        for sampled_control_index in sampled_control_indices:
            sampled_control_seqs.append(control_seqs[sampled_control_index])
            sampled_control_hgRNAs.append(control_hgRNAs[sampled_control_index])
            sampled_control_barcodes.append(control_barcodes[sampled_control_index])
            sampled_control_insert_barcodes.append(control_insert_barcodes[sampled_control_index])
            
        message_indices = range(len(message_seqs))
        sampled_message_indices = random.sample(message_indices, min(max_n_reads, len(message_indices)))
        for sampled_message_index in sampled_message_indices:
            sampled_message_seqs.append(message_seqs[sampled_message_index])
            sampled_message_hgRNAs.append(message_hgRNAs[sampled_message_index])
            sampled_message_barcodes.append(message_barcodes[sampled_message_index])
            sampled_message_insert_barcodes.append(message_insert_barcodes[sampled_message_index])
        
        seqs_dict[max_n_reads].append({'Control': sampled_control_seqs, 'Message': sampled_message_seqs})
        hgRNAs[max_n_reads].append({'Control': sampled_control_hgRNAs, 'Message': sampled_message_hgRNAs})
        barcodes[max_n_reads].append({'Control': sampled_control_barcodes, 'Message': sampled_message_barcodes})
        insert_barcodes[max_n_reads].append({'Control': sampled_control_insert_barcodes, 'Message': sampled_message_insert_barcodes})

In [15]:
len(seqs_dict[1000][2]['Control'])

1000

## LOAD BARCODES

In [16]:
barcode_df = pd.read_csv(f'{tdt_dir}/_decodingResources/BarcodeToPosition.csv')
barcode_df

,Position,Barcodes
0,1,AATTTTGCGG
1,2,TGACTTTTAA
2,3,TAACAGTATG
3,4,"TTTTTGTGAA,GTTATACTGT,CAACTCGGTC"
4,5,CAATTGTCAT
5,6,GCCGCTCAGT
6,7,TGCACGTCAT
7,8,GCGATCCCGG
8,9,AACCTAAGTT
9,10,ACTGCTCCCT


In [17]:
controls_cond_df = pd.read_csv(f'{tdt_dir}/_decodingResources/BarcodeToCondition.csv')
controls_cond_df

,Condition,Base,Barcodes
0,1,0,AATTTTGCGG
1,1,0,TGACTTTTAA
2,1,0,TAACAGTATG
3,2,g,AGTTTTTCAA
4,2,g,GGTTACACTT
5,2,g,TCTTAGCATT
6,3,gg,"TTTTTGTGAA,GTTATACTGT,CAACTCGGTC"
7,3,gg,CAATTGTCAT
8,3,gg,GCCGCTCAGT
9,4,t,TATAGCCACC


# SEQUENCE ANALYSIS

In [18]:
controls_seq_dfs = {}
message_seq_dfs = {}

for max_n_reads, n_random_states in zip(sample_max_n_reads, sample_n_random_states):
    controls_seq_dfs[max_n_reads] = []
    message_seq_dfs[max_n_reads] = []
    for rs in range(n_random_states):
        controls_seq_df = pd.DataFrame()
        controls_seq_df['Sequence'] = seqs_dict[max_n_reads][rs]['Control']
        controls_seq_df['hgRNA'] = hgRNAs[max_n_reads][rs]['Control']
        controls_seq_df['Barcode'] = insert_barcodes[max_n_reads][rs]['Control']
        controls_seq_dfs[max_n_reads].append(controls_seq_df)

        message_seq_df = pd.DataFrame()
        message_seq_df['Sequence'] = seqs_dict[max_n_reads][rs]['Message']
        message_seq_df['hgRNA'] = hgRNAs[max_n_reads][rs]['Message']
        message_seq_df['Barcode'] = insert_barcodes[max_n_reads][rs]['Message']
        message_seq_dfs[max_n_reads].append(message_seq_df)

In [19]:
controls_seq_dfs[1000][2]

,Sequence,hgRNA,Barcode
0,GACCC,D21,TGACTTTTAA
1,GGACCC,D21,TTACTTTTAA
2,G,D21,TGACTTTTAG
3,C,D21,GCGATCCCGG
4,G,D21,GTTATACTGT
...,...,...,...
995,TG,D21,GCGATCCCGG
996,G,D21,TCTTAGCATT
997,G,D21,GCGATCCCGG
998,G,D21,CAACTCGGTC


In [20]:
message_seq_dfs[1000][2]

,Sequence,hgRNA,Barcode
0,G,D21,GTTATACTGT
1,ATCCCG,D21,TCCGGACTCA
2,G,D21,GGTTACACTT
3,C,D21,TCCGGACTCA
4,G,D21,TGACTTTTAA
...,...,...,...
995,G,D21,GGTATAAAAA
996,GGGGC,D21,AACCTAAGTT
997,G,D21,TCCGGACTCA
998,A,D21,TCCGGACTCA


In [21]:
for seq_dfs in [controls_seq_dfs, message_seq_dfs]:
    for max_n_reads, n_random_states in zip(sample_max_n_reads, sample_n_random_states):
        for rs in range(n_random_states):
            df = seq_dfs[max_n_reads][rs]
            lengths = df.apply(lambda x: len(x['Sequence']), axis=1)
            df['Length'] = lengths

In [22]:
controls_seq_dfs[1000][2]

,Sequence,hgRNA,Barcode,Length
0,GACCC,D21,TGACTTTTAA,5
1,GGACCC,D21,TTACTTTTAA,6
2,G,D21,TGACTTTTAG,1
3,C,D21,GCGATCCCGG,1
4,G,D21,GTTATACTGT,1
...,...,...,...,...
995,TG,D21,GCGATCCCGG,2
996,G,D21,TCTTAGCATT,1
997,G,D21,GCGATCCCGG,1
998,G,D21,CAACTCGGTC,1


In [23]:
message_seq_dfs[1000][2]

,Sequence,hgRNA,Barcode,Length
0,G,D21,GTTATACTGT,1
1,ATCCCG,D21,TCCGGACTCA,6
2,G,D21,GGTTACACTT,1
3,C,D21,TCCGGACTCA,1
4,G,D21,TGACTTTTAA,1
...,...,...,...,...
995,G,D21,GGTATAAAAA,1
996,GGGGC,D21,AACCTAAGTT,5
997,G,D21,TCCGGACTCA,1
998,A,D21,TCCGGACTCA,1


In [24]:
controls_df = pd.DataFrame()
controls_df['Max Reads'] = [n for n, n_random_states in zip(sample_max_n_reads, sample_n_random_states)
                            for _ in range(n_random_states) ]
controls_df['Replicate'] = sum([list(range(n)) for n in sample_n_random_states], [])
controls_df['Filename'] = 'Control'
controls_df['Sample'] = 'Control'
controls_df

,Max Reads,Replicate,Filename,Sample
0,300000,0,Control,Control
1,100000,0,Control,Control
2,100000,1,Control,Control
3,100000,2,Control,Control
4,100000,3,Control,Control
...,...,...,...,...
96,100,45,Control,Control
97,100,46,Control,Control
98,100,47,Control,Control
99,100,48,Control,Control


In [25]:
message_df = pd.DataFrame()
message_df['Max Reads'] = [n for n, n_random_states in zip(sample_max_n_reads, sample_n_random_states)
                           for _ in range(n_random_states) ]
message_df['Replicate'] = sum([list(range(n)) for n in sample_n_random_states], [])
message_df['Filename'] = 'Message'
message_df['Sample'] = 'Message'
message_df

,Max Reads,Replicate,Filename,Sample
0,300000,0,Message,Message
1,100000,0,Message,Message
2,100000,1,Message,Message
3,100000,2,Message,Message
4,100000,3,Message,Message
...,...,...,...,...
96,100,45,Message,Message
97,100,46,Message,Message
98,100,47,Message,Message
99,100,48,Message,Message


In [26]:
mean_lengths = []
sd_lengths = []
counts = []

for max_n_reads, n_random_states in zip(sample_max_n_reads, sample_n_random_states):
    for rs in range(n_random_states):
        df = controls_seq_dfs[max_n_reads][rs]

        mean_length = df.Length.mean()
        sd_length = df.Length.std()
        count = len(df)

        mean_lengths.append(mean_length)
        sd_lengths.append(sd_length)
        counts.append(count)

controls_df['Avg_Length'] = mean_lengths
controls_df['Std_Length'] = sd_lengths
controls_df['Count'] = counts

controls_df

,Max Reads,Replicate,Filename,Sample,Avg_Length,Std_Length,Count
0,300000,0,Control,Control,2.495711,1.803741,152617
1,100000,0,Control,Control,2.490330,1.802348,100000
2,100000,1,Control,Control,2.495670,1.804038,100000
3,100000,2,Control,Control,2.500730,1.807726,100000
4,100000,3,Control,Control,2.499040,1.807471,100000
...,...,...,...,...,...,...,...
96,100,45,Control,Control,2.730000,2.097883,100
97,100,46,Control,Control,2.300000,1.678744,100
98,100,47,Control,Control,2.520000,1.966795,100
99,100,48,Control,Control,2.710000,1.897872,100


In [27]:
mean_lengths = []
sd_lengths = []
counts = []

for max_n_reads, n_random_states in zip(sample_max_n_reads, sample_n_random_states):
    for rs in range(n_random_states):
        df = message_seq_dfs[max_n_reads][rs]

        mean_length = df.Length.mean()
        sd_length = df.Length.std()
        count = len(df)

        mean_lengths.append(mean_length)
        sd_lengths.append(sd_length)
        counts.append(count)

message_df['Avg_Length'] = mean_lengths
message_df['Std_Length'] = sd_lengths
message_df['Count'] = counts

message_df

,Max Reads,Replicate,Filename,Sample,Avg_Length,Std_Length,Count
0,300000,0,Message,Message,2.574875,1.816339,244074
1,100000,0,Message,Message,2.573870,1.817888,100000
2,100000,1,Message,Message,2.578540,1.818033,100000
3,100000,2,Message,Message,2.574330,1.816020,100000
4,100000,3,Message,Message,2.574750,1.815358,100000
...,...,...,...,...,...,...,...
96,100,45,Message,Message,2.610000,1.582065,100
97,100,46,Message,Message,2.790000,1.887198,100
98,100,47,Message,Message,2.670000,1.907216,100
99,100,48,Message,Message,2.270000,1.562535,100


In [28]:
def count_bases(seq, base):
    count = seq.count(base)
    return count

In [29]:
for max_n_reads, n_random_states in zip(sample_max_n_reads, sample_n_random_states):
    for rs in range(n_random_states):
        df = controls_seq_dfs[max_n_reads][rs]

        # Count A, C, G, T
        for base in ['A', 'C', 'G', 'T']:
            df[base] = df.apply(lambda x: count_bases(x['Sequence'], base), axis=1)

In [30]:
for max_n_reads, n_random_states in zip(sample_max_n_reads, sample_n_random_states):
    for rs in range(n_random_states):
        df = message_seq_dfs[max_n_reads][rs]

        # Count A, C, G, T
        for base in ['A', 'C', 'G', 'T']:
            df[base] = df.apply(lambda x: count_bases(x['Sequence'], base), axis=1)

In [31]:
controls_seq_dfs[1000][2]

,Sequence,hgRNA,Barcode,Length,A,C,G,T
0,GACCC,D21,TGACTTTTAA,5,1,3,1,0
1,GGACCC,D21,TTACTTTTAA,6,1,3,2,0
2,G,D21,TGACTTTTAG,1,0,0,1,0
3,C,D21,GCGATCCCGG,1,0,1,0,0
4,G,D21,GTTATACTGT,1,0,0,1,0
...,...,...,...,...,...,...,...,...
995,TG,D21,GCGATCCCGG,2,0,0,1,1
996,G,D21,TCTTAGCATT,1,0,0,1,0
997,G,D21,GCGATCCCGG,1,0,0,1,0
998,G,D21,CAACTCGGTC,1,0,0,1,0


In [32]:
message_seq_dfs[1000][2]

,Sequence,hgRNA,Barcode,Length,A,C,G,T
0,G,D21,GTTATACTGT,1,0,0,1,0
1,ATCCCG,D21,TCCGGACTCA,6,1,3,1,1
2,G,D21,GGTTACACTT,1,0,0,1,0
3,C,D21,TCCGGACTCA,1,0,1,0,0
4,G,D21,TGACTTTTAA,1,0,0,1,0
...,...,...,...,...,...,...,...,...
995,G,D21,GGTATAAAAA,1,0,0,1,0
996,GGGGC,D21,AACCTAAGTT,5,0,1,4,0
997,G,D21,TCCGGACTCA,1,0,0,1,0
998,A,D21,TCCGGACTCA,1,1,0,0,0


In [33]:
controls_bc_dfs = {}
filename = 'Control'

for max_n_reads, n_random_states in zip(sample_max_n_reads, sample_n_random_states):
    controls_bc_dfs[max_n_reads] = []
    for rs in range(n_random_states):
        controls_bc_df = pd.DataFrame()

        all_filenames = []
        all_barcodes = []
        all_conditions = []
        all_a_pcts = []
        all_c_pcts = []
        all_g_pcts = []
        all_t_pcts = []
        all_lengths = []

        for df in [controls_seq_dfs[max_n_reads][rs]]:
            filenames = []
            sub_barcodes = []
            conditions = []
            a_pcts = []
            c_pcts = []
            g_pcts = []
            t_pcts = []
            lengths = []

            for file_barcodes, condition in zip(controls_cond_df.Barcodes, controls_cond_df.Condition):
                if ',' in file_barcodes:
                    file_barcodes = file_barcodes.split(',')
                else:
                    file_barcodes = [file_barcodes]

                a = 0
                c = 0
                g = 0
                t = 0
                n_seqs = 0

                for barcode in file_barcodes:
                    sub_df = df.loc[df.Barcode == barcode]

                    a += sub_df.A.sum()
                    c += sub_df.C.sum()
                    g += sub_df.G.sum()
                    t += sub_df['T'].sum()  # .T is transpose
                    n_seqs += len(sub_df)

                if not a:
                    a = 1
                if not c:
                    c = 1
                if not g:
                    g = 1
                if not t:
                    t = 1
                if not n_seqs:
                    n_seqs = 999999999

                total = a + c + g + t
                avg_len = total / n_seqs

                filenames.append(filename)
                sub_barcodes.append(','.join(file_barcodes))
                conditions.append(condition)
                a_pcts.append(a / total)
                c_pcts.append(c / total)
                g_pcts.append(g / total)
                t_pcts.append(t / total)
                lengths.append(avg_len)

            all_filenames += filenames
            all_barcodes += sub_barcodes
            all_conditions += conditions
            all_a_pcts += a_pcts
            all_c_pcts += c_pcts
            all_g_pcts += g_pcts
            all_t_pcts += t_pcts
            all_lengths += lengths

        controls_bc_df['Filename'] = all_filenames
        controls_bc_df['Barcodes'] = all_barcodes
        controls_bc_df['Condition'] = all_conditions
        controls_bc_df['A%'] = all_a_pcts
        controls_bc_df['C%'] = all_c_pcts
        controls_bc_df['G%'] = all_g_pcts
        controls_bc_df['T%'] = all_t_pcts
        controls_bc_df['Length'] = all_lengths
        
        controls_bc_dfs[max_n_reads].append(controls_bc_df)

In [34]:
controls_bc_dfs[1000][2]

,Filename,Barcodes,Condition,A%,C%,G%,T%,Length
0,Control,AATTTTGCGG,1,0.161290,0.215054,0.564516,0.059140,2.325000e+00
1,Control,TGACTTTTAA,1,0.144654,0.220126,0.509434,0.125786,2.373134e+00
2,Control,TAACAGTATG,1,0.160714,0.232143,0.500000,0.107143,2.800000e+00
3,Control,AGTTTTTCAA,2,0.116667,0.277778,0.538889,0.066667,2.432432e+00
4,Control,GGTTACACTT,2,0.090909,0.278409,0.551136,0.079545,2.588235e+00
5,Control,TCTTAGCATT,2,0.123810,0.361905,0.457143,0.057143,2.386364e+00
6,Control,"TTTTTGTGAA,GTTATACTGT,CAACTCGGTC",3,0.020833,0.416667,0.531250,0.031250,2.341463e+00
7,Control,CAATTGTCAT,3,0.038168,0.320611,0.633588,0.007634,2.471698e+00
8,Control,GCCGCTCAGT,3,0.027523,0.211009,0.715596,0.045872,2.137255e+00
9,Control,TATAGCCACC,4,0.158730,0.275132,0.460317,0.105820,2.661972e+00


In [35]:
message_bc_dfs = {}
filename = 'Message'

for max_n_reads, n_random_states in zip(sample_max_n_reads, sample_n_random_states):
    message_bc_dfs[max_n_reads] = []
    for rs in range(n_random_states):
        message_bc_df = pd.DataFrame()

        all_filenames = []
        all_barcodes = []
        all_positions = []
        all_a_pcts = []
        all_c_pcts = []
        all_g_pcts = []
        all_t_pcts = []
        all_lengths = []

        for df in [message_seq_dfs[max_n_reads][rs]]:
            filenames = []
            sub_barcodes = []
            positions = []
            a_pcts = []
            c_pcts = []
            g_pcts = []
            t_pcts = []
            lengths = []

            for file_barcodes, position in zip(barcode_df.Barcodes, barcode_df.Position):
                if ',' in file_barcodes:
                    file_barcodes = file_barcodes.split(',')
                else:
                    file_barcodes = [file_barcodes]

                a = 0
                c = 0
                g = 0
                t = 0
                n_seqs = 0
                for barcode in file_barcodes:
                    sub_df = df.loc[df.Barcode == barcode]

                    a += sub_df.A.sum()
                    c += sub_df.C.sum()
                    g += sub_df.G.sum()
                    t += sub_df['T'].sum()
                    n_seqs += len(sub_df)

                if not a:
                    a = 1
                if not c:
                    c = 1
                if not g:
                    g = 1
                if not t:
                    t = 1
                if not n_seqs:
                    n_seqs = 99999999  # To get 0 for avg length

                total = a + c + g + t
                avg_len = total / n_seqs

                filenames.append(filename)
                sub_barcodes.append(file_barcodes)
                positions.append(position)
                a_pcts.append(a / total)
                c_pcts.append(c / total)
                g_pcts.append(g / total)
                t_pcts.append(t / total)
                lengths.append(avg_len)

            all_filenames += filenames
            all_barcodes += sub_barcodes
            all_positions += positions
            all_a_pcts += a_pcts
            all_c_pcts += c_pcts
            all_g_pcts += g_pcts
            all_t_pcts += t_pcts
            all_lengths += lengths

        message_bc_df['Filename'] = all_filenames
        message_bc_df['Barcodes'] = all_barcodes
        message_bc_df['Position'] = all_positions
        message_bc_df['A%'] = all_a_pcts
        message_bc_df['C%'] = all_c_pcts
        message_bc_df['G%'] = all_g_pcts
        message_bc_df['T%'] = all_t_pcts
        message_bc_df['Length'] = all_lengths
        
        message_bc_dfs[max_n_reads].append(message_bc_df)

In [36]:
message_bc_dfs[1000][2]

,Filename,Barcodes,Position,A%,C%,G%,T%,Length
0,Message,[AATTTTGCGG],1,0.062500,0.437500,0.437500,0.062500,3.200000
1,Message,[TGACTTTTAA],2,0.048387,0.370968,0.540323,0.040323,2.254545
2,Message,[TAACAGTATG],3,0.136842,0.252632,0.505263,0.105263,2.317073
3,Message,"[TTTTTGTGAA, GTTATACTGT, CAACTCGGTC]",4,0.105263,0.254386,0.394737,0.245614,2.533333
4,Message,[CAATTGTCAT],5,0.070796,0.256637,0.610619,0.061947,2.132075
5,Message,[GCCGCTCAGT],6,0.125000,0.125000,0.625000,0.125000,2.666667
6,Message,[TGCACGTCAT],7,0.069444,0.333333,0.513889,0.083333,2.571429
7,Message,[GCGATCCCGG],8,0.250000,0.250000,0.250000,0.250000,4.000000
8,Message,[AACCTAAGTT],9,0.021898,0.467153,0.503650,0.007299,3.512821
9,Message,[ACTGCTCCCT],10,0.023622,0.330709,0.598425,0.047244,2.760870


In [37]:
for max_n_reads, n_random_states in zip(sample_max_n_reads, sample_n_random_states):
    for rs in range(n_random_states):
        aitch_pcts = pd.DataFrame(clr(controls_bc_dfs[max_n_reads][rs].loc[:, ['A%', 'C%', 'G%', 'T%']]),
                                  columns=['A_aitch', 'C_aitch', 'G_aitch', 'T_aitch'])
        controls_bc_dfs[max_n_reads][rs] = pd.concat([controls_bc_dfs[max_n_reads][rs], aitch_pcts], axis=1)

In [38]:
controls_bc_dfs[1000][2]

,Filename,Barcodes,Condition,A%,C%,G%,T%,Length,A_aitch,C_aitch,G_aitch,T_aitch
0,Control,AATTTTGCGG,1,0.161290,0.215054,0.564516,0.059140,2.325000e+00,-0.134286,0.153396,1.118477,-1.137588
1,Control,TGACTTTTAA,1,0.144654,0.220126,0.509434,0.125786,2.373134e+00,-0.384762,0.035092,0.874193,-0.524524
2,Control,TAACAGTATG,1,0.160714,0.232143,0.500000,0.107143,2.800000e+00,-0.274310,0.093415,0.860670,-0.679775
3,Control,AGTTTTTCAA,2,0.116667,0.277778,0.538889,0.066667,2.432432e+00,-0.459518,0.407982,1.070670,-1.019134
4,Control,GGTTACACTT,2,0.090909,0.278409,0.551136,0.079545,2.588235e+00,-0.696956,0.422276,1.105167,-0.830487
5,Control,TCTTAGCATT,2,0.123810,0.361905,0.457143,0.057143,2.386364e+00,-0.401425,0.671212,0.904827,-1.174615
6,Control,"TTTTTGTGAA,GTTATACTGT,CAACTCGGTC",3,0.020833,0.416667,0.531250,0.031250,2.341463e+00,-1.659969,1.335763,1.578709,-1.254504
7,Control,CAATTGTCAT,3,0.038168,0.320611,0.633588,0.007634,2.471698e+00,-0.832049,1.296183,1.977354,-2.441487
8,Control,GCCGCTCAGT,3,0.027523,0.211009,0.715596,0.045872,2.137255e+00,-1.451451,0.585431,1.806646,-0.940625
9,Control,TATAGCCACC,4,0.158730,0.275132,0.460317,0.105820,2.661972e+00,-0.302323,0.247723,0.762388,-0.707788


In [39]:
for max_n_reads, n_random_states in zip(sample_max_n_reads, sample_n_random_states):
    for rs in range(n_random_states):
        aitch_pcts = pd.DataFrame(clr(message_bc_dfs[max_n_reads][rs].loc[:, ['A%', 'C%', 'G%', 'T%']]),
                                  columns=['A_aitch', 'C_aitch', 'G_aitch', 'T_aitch'])
        message_bc_dfs[max_n_reads][rs] = pd.concat([message_bc_dfs[max_n_reads][rs], aitch_pcts], axis=1)

In [40]:
message_bc_dfs[1000][2]

,Filename,Barcodes,Position,A%,C%,G%,T%,Length,A_aitch,C_aitch,G_aitch,T_aitch
0,Message,[AATTTTGCGG],1,0.062500,0.437500,0.437500,0.062500,3.200000,-0.972955,0.972955,0.972955,-0.972955
1,Message,[TGACTTTTAA],2,0.048387,0.370968,0.540323,0.040323,2.254545,-1.066873,0.970009,1.346060,-1.249195
2,Message,[TAACAGTATG],3,0.136842,0.252632,0.505263,0.105263,2.317073,-0.414248,0.198857,0.892004,-0.676612
3,Message,"[TTTTTGTGAA, GTTATACTGT, CAACTCGGTC]",4,0.105263,0.254386,0.394737,0.245614,2.533333,-0.762861,0.119528,0.558895,0.084437
4,Message,[CAATTGTCAT],5,0.070796,0.256637,0.610619,0.061947,2.132075,-0.827247,0.460607,1.327418,-0.960778
5,Message,[GCCGCTCAGT],6,0.125000,0.125000,0.625000,0.125000,2.666667,-0.402359,-0.402359,1.207078,-0.402359
6,Message,[TGCACGTCAT],7,0.069444,0.333333,0.513889,0.083333,2.571429,-0.938104,0.630512,1.063376,-0.755783
7,Message,[GCGATCCCGG],8,0.250000,0.250000,0.250000,0.250000,4.000000,0.000000,0.000000,0.000000,0.000000
8,Message,[AACCTAAGTT],9,0.021898,0.467153,0.503650,0.007299,3.512821,-1.274288,1.785983,1.861206,-2.372900
9,Message,[ACTGCTCCCT],10,0.023622,0.330709,0.598425,0.047244,2.760870,-1.641081,0.997976,1.591040,-0.947934


# COMBINE INTO ONE BIG DATAFRAME

In [41]:
full_df = pd.DataFrame()

for df_set in [controls_bc_dfs, message_bc_dfs]:
    for max_n_reads, n_random_states in zip(sample_max_n_reads, sample_n_random_states):
        for rs in range(n_random_states):
            df_set[max_n_reads][rs]['Max Reads'] = max_n_reads
            df_set[max_n_reads][rs]['Replicate'] = rs
            full_df = pd.concat([full_df, df_set[max_n_reads][rs]])

for column, column_i in zip(['Max Reads', 'Replicate', 'Position'], [1, 2, 5]):
    vals = full_df[column]
    full_df = full_df.drop([column], axis=1)
    full_df.insert(loc=column_i, column=column, value=vals)

In [42]:
full_df

,Filename,Max Reads,Replicate,Barcodes,Condition,Position,A%,C%,G%,T%,Length,A_aitch,C_aitch,G_aitch,T_aitch
0,Control,300000,0,AATTTTGCGG,1.0,NaN,0.135076,0.237882,0.558091,0.068951,2.380663e+00,-0.328049,0.237891,1.090640,-1.000483
1,Control,300000,0,TGACTTTTAA,1.0,NaN,0.108198,0.259366,0.535040,0.097395,2.245217e+00,-0.591864,0.282411,1.006511,-0.697057
2,Control,300000,0,TAACAGTATG,1.0,NaN,0.110600,0.273327,0.520516,0.095557,2.466897e+00,-0.576861,0.327885,0.972037,-0.723061
3,Control,300000,0,AGTTTTTCAA,2.0,NaN,0.084250,0.271443,0.570239,0.074067,2.378713e+00,-0.738354,0.431611,1.173913,-0.867170
4,Control,300000,0,GGTTACACTT,2.0,NaN,0.084703,0.273646,0.573650,0.068002,2.475653e+00,-0.716483,0.456200,1.196386,-0.936102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,Message,100,49,[CAATAACCGA],NaN,20.0,0.250000,0.250000,0.250000,0.250000,4.000000e-08,0.000000,0.000000,0.000000,0.000000
20,Message,100,49,[TTCAATTCGA],NaN,21.0,0.200000,0.100000,0.600000,0.100000,1.666667e+00,0.071921,-0.621227,1.170533,-0.621227
21,Message,100,49,[TATAGCCACC],NaN,22.0,0.125000,0.250000,0.500000,0.125000,2.666667e+00,-0.519860,0.173287,0.866434,-0.519860
22,Message,100,49,[CCTTCGACTC],NaN,23.0,0.133333,0.333333,0.400000,0.133333,3.750000e+00,-0.503726,0.412565,0.594887,-0.503726


In [43]:
full_df.to_csv(figure_dir + '/full_subsampled_metrics.csv')